### This notebook runs the dimensional load for all dims from sybase.

In [ ]:
%run SLV_NB_Functions

StatementMeta(, f12487cd-84ed-4c68-b0ab-2929d9eb9ae9, 3, Finished, Available, Finished)

# Dim Supplier Section

In [ ]:
fproveedores_df = spark.read.parquet(get_latest_file('fproveedores')) ##supplier 

fproveedores_df.createOrReplaceTempView("Supplier")

StatementMeta(, d66e89fa-c365-40d5-87bc-3b3d61486efc, 6, Finished, Available, Finished)

In [6]:
 from pyspark.sql import SparkSession
 from pyspark.sql.functions import col, lit, expr, date_format, dayofweek, weekofyear, month, quarter, year, dayofmonth, dayofyear, last_day, when

# Define the date range
# start_date = "2015-01-01"
# end_date = "2050-12-31"

 # Generate a Date Sequence
 date_df = spark.sql(f"SELECT explode(sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day)) as date")

# # Create DimDate attributes
 dim_date_df = date_df.select(
     col("date").alias("full_date"),
     year("date").alias("year"),
     quarter("date").alias("quarter"),
     month("date").alias("month"),
     date_format("date", "MMMM").alias("month_name"),
     date_format("date", "E").alias("day_name"),
     dayofmonth("date").alias("day_of_month"),
     dayofweek("date").alias("day_of_week"),  # 1=Sunday, 7=Saturday
     dayofyear("date").alias("day_of_year"),
     weekofyear("date").alias("week_of_year"),
     when((dayofweek("date") == 1) | (dayofweek("date") == 7), lit(1)).otherwise(lit(0)).alias("is_weekend"),
     expr("CASE WHEN month(date) IN (1, 2, 3) THEN 'Q1' WHEN month(date) IN (4, 5, 6) THEN 'Q2' WHEN month(date) IN (7, 8, 9) THEN 'Q3' ELSE 'Q4' END").alias("quarter_name"),
     expr("CASE WHEN dayofweek(date) IN (1,7) THEN 'Weekend' ELSE 'Weekday' END").alias("day_type"),
 )

 dim_date_df.write.mode('overwrite').saveAsTable('DimDates')


StatementMeta(, f06bedf1-1a28-4a2b-9697-cf24d681d416, 8, Finished, Available, Finished)